# Zero to Snowflake - AISQL Functions

```
Asset:        Zero to Snowflake - AISQL Functions
Version:      v1     
Copyright(c): 2025 Snowflake Inc. All rights reserved.
```

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczNQXvH9nD0-85OIo1jjfar5VYUjmTG7wpls3voC4zFmXcRTpg4QwxZxyOCtsLCNbcIuHLgSBGv-x7vEkMnTAirhXYHeUPNK9giZvQFNIZJQOzUFud1wQ1eRS158qGBTkBtJI54Yc9Yhpmg86Il4H8OB=w1500-h844-s-no-gm?authuser=0)

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczNm8a6UL-SVn04eK7EmXaB9iVaNefQ4fUpYrSFScypSQA0Mc_wv4sb9uFFCqRbyWeMIf-Gcpk4oC4IJVMPkqPYx5trw3fnREAsTxR5Ny_Goy8hGCatMUplj5rRAQv9mUuFLbg1yq21fm6369FFWQ9Sh=w1500-h844-s-no-gm?authuser=0)

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczMU3QqLcSuGc0fAohLN08qjHKNcZhefDR00974EqkaoSqWK2-d2IEsaiz4trhLEfw_MR94UIYImWEllX0BCQnn1A5j0t3sL-D3QoBwXQIzDOYYKy4wTD_l-54N0soNBTYOy6LrvBej8MB0qgGzoVbs5=w1500-h844-s-no-gm?authuser=0)

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczN8jwbemv9nmNwIphkV5cKcSy9mNCHlfjTdDNwzFXG5QP9OBVLsUQ--iuAJZiH-lfkdTmNU-cyG8WiPM3nhuvgyjx6YY8_m1YovtlWGOno7I1LMXmnzjl0xhBoazyewLeT-2PptG8TKoJzUr1oEADW_=w1500-h844-s-no-gm?authuser=0)

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczNlcULLcw7vXRzWZnTQSKpaZcPrbaTilaeNn1fn8h6we5R5ehRKo70jvGrDz05Fx3l-NiYGMLMWV_vpe-ha35udAhQ1lIHblpFsr1VNVvr5qEJUeIR5Rc-maKCwxSqUcPXA0k5tXXdtbOA_EakKWYvl=w1500-h844-s-no-gm?authuser=0)

## 目次
1. SENTIMENT()を使用してトラックの顧客レビューをポジティブ、ネガティブ、ニュートラルとしてスコア化・ラベル付け
2. AI_CLASSIFY()を使用して食品品質やサービス体験などのテーマでレビューを分類
3. EXTRACT_ANSWER()を使用してレビューテキストから特定の苦情や称賛を抽出
4. AI_SUMMARIZE_AGG()を使用してトラックブランド名ごとの顧客センチメントの簡潔な要約を生成


In [ ]:
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_101_v2_aisql","version":{"major":1, "minor":1},"attributes":{"is_quickstart":0, "source":"tastybytes", "vignette": "aisql_functions"}}';


顧客レビューからインサイトを得るためにAISSQL関数を活用することを意図したTasteBytesデータアナリストの役割を担うので、適切にコンテキストを設定しましょう。


In [ ]:
USE ROLE tb_analyst;
USE DATABASE tb_101;
USE WAREHOUSE tb_analyst_wh;


## 1. 大規模なセンチメント分析

すべてのフードトラックブランドの顧客センチメントを分析して、どのトラックが最高のパフォーマンスを発揮しているかを特定し、車両群全体の顧客満足度指標を作成します。Cortex Playgroundでは、個々のレビューを手動で分析しました。今度はSENTIMENT()関数を使用して、-1（ネガティブ）から+1（ポジティブ）まで、Snowflakeの公式センチメント範囲に従って顧客レビューを自動的にスコア化します。


**ビジネス質問：** 「各トラックブランドに対して顧客は全体的にどのように感じているか？」

このクエリを実行してフードトラックネットワーク全体の顧客センチメントを分析し、フィードバックを分類してください


In [ ]:
SELECT
    truck_brand_name,
    COUNT(*) AS total_reviews,
    AVG(CASE WHEN sentiment >= 0.5 THEN sentiment END) AS avg_positive_score,
    AVG(CASE WHEN sentiment BETWEEN -0.5 AND 0.5 THEN sentiment END) AS avg_neutral_score,
    AVG(CASE WHEN sentiment <= -0.5 THEN sentiment END) AS avg_negative_score
FROM (
    SELECT
        truck_brand_name,
        SNOWFLAKE.CORTEX.SENTIMENT (review) AS sentiment
    FROM harmonized.truck_reviews_v
    WHERE
        language ILIKE '%en%'
        AND review IS NOT NULL
    LIMIT 10000
)
GROUP BY
    truck_brand_name
ORDER BY total_reviews DESC;


### 主要なインサイト：
Cortex Playgroundで一度に一つずつレビューを分析していたものから、数千件を体系的に処理することへの移行に注目してください。SENTIMENT()関数はすべてのレビューを自動的にスコア化し、ポジティブ、ネガティブ、ニュートラルに分類し、即座に車両群全体の顧客満足度指標を提供しました。

### センチメントスコア範囲：
- **ポジティブ:** 0.5 から 1
- **ニュートラル:** -0.5 から 0.5
- **ネガティブ:** -0.5 から -1


### 参考:AI_SENTIMENT 関数

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczOl-mGz0N_RmsJ7ZG3Cl5j7NmuMe1A84rvLEr3Wt0sTbJcAOv0mm9fdAPlUYW-MwkvVfsveedNSCKADtk9C4rgZ7i5WqZoXRP1JbbOxgcrvp1qfSoQWgKqKd0PpXySMxQQPWtPyBULmBUGQ1ORppLda=w1500-h844-s-no-gm?authuser=0)

AI_SENTIMENT は SENTIMENT （SNOWFLAKE.CORTEX） 関数の更新されたバージョンです。


In [ ]:
SELECT
    truck_brand_name,
    sentiment:categories[0].sentiment
FROM (
    SELECT
        truck_brand_name,
        AI_SENTIMENT (review) AS sentiment
    FROM harmonized.truck_reviews_v
    WHERE
        language ILIKE '%en%'
        AND review IS NOT NULL
    LIMIT 10
)

## 2. 顧客フィードバックの分類

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczMXZhDTAurJJ0kwP04iOzpDq4TrfHexI5xT_KvOKzeFxStxGrIPtLetSUDV0MbtBMxEsyKveDkNjPwcAwuYnNh4WV-d0Q1qmalNyBry4rAxOZBS9LPs3m4E7GtTKRTb-UWlpqpTBTm6AJyBYSmzlmqL=w1500-h844-s-no-gm?authuser=0)

次に、すべてのレビューを分類して、顧客がサービスのどの側面について最も多く話しているかを理解しましょう。単純なキーワードマッチングではなく、AI理解に基づいてレビューをユーザー定義カテゴリに自動的に分類するAI_CLASSIFY()関数を使用します。この段階では、顧客フィードバックをビジネス関連の運営分野に分類し、それらの分布パターンを分析します。


**ビジネス質問：** 「顧客は主に何についてコメントしているか - 食品品質、サービス、配達体験？」

分類クエリを実行：


In [ ]:
WITH classified_reviews AS (
  SELECT
    truck_brand_name,
    AI_CLASSIFY(
      review,
      ['Food Quality', 'Pricing', 'Service Experience', 'Staff Behavior']
    ):labels[0] AS feedback_category
  FROM
    harmonized.truck_reviews_v
  WHERE
    language ILIKE '%en%'
    AND review IS NOT NULL
    AND LENGTH(review) > 30
  LIMIT
    10000
)
SELECT
  truck_brand_name,
  feedback_category,
  COUNT(*) AS number_of_reviews
FROM
  classified_reviews
GROUP BY
  truck_brand_name,
  feedback_category
ORDER BY
  truck_brand_name,
  number_of_reviews DESC;


### 主要なインサイト：
AI_CLASSIFY()が数千のレビューを食品品質、サービス体験などのビジネス関連テーマに自動的に分類したことを観察してください。食品品質がトラックブランド全体で最も議論されるトピックであることがすぐにわかり、運営チームに顧客の優先事項についての明確で実行可能なインサイトを提供します。


## 3. 特定の運営インサイトの抽出

次に、非構造化テキストから正確な回答を得るために、EXTRACT_ANSWER()関数を活用します。この強力な関数により、顧客フィードバックについて特定のビジネス質問を投げかけ、直接的な回答を受け取ることができます。この段階では、顧客レビューで言及されている正確な運営上の問題を特定し、即座の注意が必要な特定の問題を強調することが目標です。


**ビジネス質問：** 「各顧客レビュー内で言及されている特定の運営上の問題やポジティブな言及は何か？」

次のクエリを実行しましょう：


In [ ]:
SELECT
    truck_brand_name,
    review
FROM 
    harmonized.truck_reviews_v
LIMIT 10

In [ ]:
  SELECT
    truck_brand_name,
    primary_city,
    LEFT(review, 100) || '...' AS review_preview,
    SNOWFLAKE.CORTEX.EXTRACT_ANSWER(
        review,
        'What specific improvement or complaint is mentioned in this review?'
    ) AS specific_feedback
FROM 
    harmonized.truck_reviews_v
WHERE 
    language = 'en'
    AND review IS NOT NULL
    AND LENGTH(review) > 50
ORDER BY truck_brand_name, primary_city ASC
LIMIT 10000;


### 主要なインサイト：
EXTRACT_ANSWER()が長い顧客レビューから具体的で実行可能なインサイトを抽出していることに注目してください。手動レビューではなく、この関数は「親切なスタッフが救いだった」や「ホットドッグは完璧に調理されている」などの具体的なフィードバックを自動的に特定します。結果は、密なテキストを運営チームが即座に活用できる具体的で引用可能なフィードバックに変換することです。


## 4. エグゼクティブサマリーの生成

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczPB6NYcPP1mV6kEyK--xFlMcxI6GMA99NKKNEy6y70QHrZNyYP5aE-_guGSWshdoJ_biFVvL9RQ9F3IhSpxOdRmZAD9u0bqRru7GamyCbIB6072KSrHEUPp7Q4aQiw4NtR4-M5cl_H0JcpIsXQAXInO=w1500-h844-s-no-gm?authuser=0)

最後に、顧客フィードバックの簡潔な要約を作成するために、AI_SUMMARIZE_AGG()関数を使用します。この強力な関数は、長い非構造化テキストから短く一貫した要約を生成します。この段階では、各トラックブランドの顧客レビューの本質を消化しやすい要約に凝縮し、全体的なセンチメントとキーポイントの簡潔な概要を提供することが目標です。


**ビジネス質問：** 「各トラックブランドのキーテーマと全体的なセンチメントは何か？」

要約クエリを実行：


In [ ]:
SELECT
  truck_brand_name,
  AI_SUMMARIZE_AGG (review) AS review_summary
FROM
  (
    SELECT
      truck_brand_name,
      review
    FROM
      harmonized.truck_reviews_v
    LIMIT
      100
  )
GROUP BY
  truck_brand_name;


### 主要なインサイト：
AI_SUMMARIZE_AGG()関数は、長いレビューを明確なブランドレベルの要約に凝縮します。これらの要約は、反復テーマとセンチメントトレンドを強調し、決定者に各フードトラックのパフォーマンスの簡潔な概要を提供し、個々のレビューを読むことなく顧客認識をより迅速に理解できるようにします。


---

## まとめ

![IMAGES](https://lh3.googleusercontent.com/pw/AP1GczMLh9iSt6uWid0mD34viEtp3zvsf1qSiM8G41rjUrhHvmYw2v4epHmtMMaTA_gzu34iw1Sm4hqM3t02Imm7NKu8zTdhBeSBlvwfOP9qGzIqN5g8t6D1gr1OE4xDXa7sb2_jWN8bIKWTP8d1hcjdLmuq=w1500-h844-s-no-gm?authuser=0)

私たちは、AI SQL関数の変革的な力を成功裏に実証し、顧客フィードバック分析を個々のレビュー処理から体系的で本格的な規模のインテリジェンスへと移行させました。これら4つのコア関数を通じた私たちの旅路は、それぞれが明確な分析目的を果たし、生の顧客の声を包括的なビジネスインテリジェンスに変換することを明確に示しています—体系的で、スケーラブルで、即座に実行可能です。かつて個々のレビュー分析を必要としていたものが、今では数秒で数千のレビューを処理し、データ駆動型の運営改善に不可欠な感情的コンテキストと具体的な詳細の両方を提供します。

分析がより洗練されるにつれて、複雑なクエリで複数のAI関数を組み合わせることが困難になる場合があります。これはまさにSnowflake Copilotが強力な支援を提供する場面です。Copilotが自然言語を使用して複雑なクエリを作成し、作業を大幅に加速する方法を探ってみましょう。これにより、より複雑な分析ワークフローを簡単に構築できるようになります。
